In [45]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import sqlite3
import SniperUtils as SU
from src.core import IterUtils
from src.view import DataView as dview

In [54]:
print([1,2,3,4,5][1:])
print(pd.Timestamp('2020-07-04'))

[2, 3, 4, 5]
2020-07-04 00:00:00


In [2]:
def ls_win_constr(min_t):
    return f'lsn_win_{min_t.microseconds}us'

def tick_constr(min_t):
    return f'entry_ticks_{min_t}'

def default_ls_win_constrs():
    return [pd.Timedelta(us, unit='us') for us in [55, 75, 100, 150, 200]]

def default_tick_constrs():
    return [1, 2, 3, 4, 5]

def add_constrs(df: pd.DataFrame, ls_win_constrs = default_ls_win_constrs(), tick_constrs = default_tick_constrs()):
    for t in ls_win_constrs:
        df[ls_win_constr(t)] = df['lsn_win'] >= t
    for t in tick_constrs:
        df[tick_constr(t)] = df['entry_ticks'] >= t
    return df

def remove_double_dips(df):
    return df
    # def max_edge (opp_rows):
    #     m_idx = opp_rows['tot_ticks'].argmax()
    #     r = opp_rows.iloc[m_idx]
    #     return r
    # res = df.groupby(['opp_id', 'market_date']).apply(lambda x: max_edge(x))
    # return res

In [3]:
sql_conn = sqlite3.connect('/md/SIM_DBs/sim_zs.sqlite')
root_df = pd.read_sql('select * from sniper_opps', sql_conn)
df = SU.cast_sniper_opps_cols(root_df)
df['tot_ticks'] = df['entry_ticks'] * df['entry_qty']
df = SU.add_constrs(df)
df = SU.remove_double_dips(df)

In [39]:
df['lsn_win_us'] = df['lsn_win'].apply(lambda x: x.microseconds)
qty_grps = df[['entry_qty', 'lsn_win_us']].groupby('entry_qty')
quantiles = [.1, .2, .3, .5, .7, .8, .9]
def get_quantiles(g):
    row = []
    for q in quantiles:
        row.append(g['lsn_win'].quantile(q))
    return row

qt1=qty_grps.quantile(.1)
qt1.style.format({'lsn_win_us': lambda x: f'{x:.2f}'})
#     return [g['lsn_win'].quantile(q) for q in quantiles]
# l = qty_grps.apply(get_quantiles)
# pd.DataFrame(data=[l], columns=[str(q) for q in quantiles])
# pd.DataFrame(data=qty_grps.apply(get_quantiles), columns=[str(q) for q in quantiles])
# pd.DataFrame(data=qty_grps.apply(get_quantiles), columns=[str(q) for q in quantiles])
# for grp in qty_grps.groups:
#     qty_grps.get_group(grp)
#     break

,lsn_win_us
entry_qty,
0,0.00
1,62.00
2,58.00
3,45.00
4,73.00
5,57.20
6,50.00
7,77.30
8,39.00


In [44]:
qts = [qty_grps.quantile(q) for q in quantiles]
summ_data = {}
qt_idx = 0
for q in quantiles:
    summ_data[f'{q:.2f}'] = qts[qt_idx]['lsn_win_us']
    qt_idx += 1
pd.DataFrame(data=summ_data)
# summ = pd.DataFrame(data={'0.1': qts[0]['lsn_win_us']})
# summ

,0.10,0.20,0.30,0.50,0.70,0.80,0.90
entry_qty,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,62.0,156.2,3349.1,983876.0,996152.0,998352.4,999600.0
2,58.0,94.6,779.8,983305.5,997232.2,999180.2,999853.4
3,45.0,84.6,173.2,25796.0,997502.6,999386.0,999849.6
4,73.0,102.0,169.0,14455.0,997822.8,999529.0,999881.8
5,57.2,92.2,247.6,989653.0,998340.2,999490.4,999871.9
6,50.0,123.0,509.0,34601.5,998807.5,999668.0,999933.5
7,77.3,152.0,1179.9,996331.5,999336.8,999782.4,999964.6
8,39.0,78.0,107.0,3371.0,998371.5,999649.0,999889.5


In [27]:
df = SU.cast_sniper_opps_cols(root_df)
# jun = df.loc[df['market_date'] >= pd.Timestamp(2021, 6, 1)]
min_t = pd.Timedelta(55, unit='us')
add_ls_win_constr(df, [min_t])
df.loc[df[ls_win_constr(min_t)]]
per_poly = df.groupby('poly')
def sum_n_opps(x):
    constr=ls_win_constr(min_t)
    good = x.loc[(x[constr]) & (x['entry_qty'] >= 5)]
    # bad = x.loc[!(x[fmt_ls_win_constr(min_t)])]
    return len(good)
rows=per_poly.apply(sum_n_opps)
res=pd.DataFrame(data=rows, columns=['n_good'])
# res_srt=res.loc[res['entry_ticks'] > 1]
res_srt=res.sort_values(by='n_good', ascending=False)
display(res_srt.head(20))
print(res['n_good'].sum())
# jun.loc[jun['lsn_win'] >= pd.Timedelta(55, unit='us')]
# jun.info()

,n_good
poly,
+(ZSN1-ZSQ1) -(ZSN1-ZSX1) +(ZSQ1-ZSX1),48
+(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-ZSX1),24
+(ZSN1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-ZSX1),16
+(ZSN1-ZSQ1) -(ZSN1-ZSU1) +(ZSQ1-ZSU1),13
+(ZSN1-ZSQ1) -(ZSN1-ZSX1) +(ZS:BF Q1-U1-X1) +2(ZSU1-ZSX1),13
+(ZSH1-ZSK1) -(ZSH1-ZSN1) +(ZSK1-ZSN1),12
+(ZSK1-ZSN1) -(ZSK1-ZSQ1) +(ZSN1-ZSQ1),11
+3(ZSK1-ZSN1) -2(ZSK1-ZSQ1) -(ZS:BF K1-N1-Q1) +(ZSN1-ZSQ1),11
+ZSK1 -(ZSK1-ZSN1) -ZSN1,11


903


In [7]:
# zs = get_opps('/home/taha/spartan_store/sniper_opp/sim_gc_complete.csv', 10,1)
# opps = get_opps(root_df, 5, 10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123761 entries, 0 to 123760
Data columns (total 36 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   market_date      123761 non-null  object
 1   run_id           123761 non-null  int64 
 2   opp_id           123761 non-null  object
 3   eid              123761 non-null  object
 4   ht_time          123761 non-null  object
 5   t_time           123615 non-null  object
 6   side             123615 non-null  object
 7   poly             123615 non-null  object
 8   has_futures      123615 non-null  object
 9   min_days         123615 non-null  object
 10  max_days         123615 non-null  object
 11  min_fut_vol      123615 non-null  object
 12  entry_ticks      123615 non-null  object
 13  entry_pnl        123615 non-null  object
 14  entry_qty        123615 non-null  object
 15  is_direct        123615 non-null  object
 16  fsn_win          123615 non-null  object
 17  lsn_win   